<p align="center">
  <a href="" rel="noopener">
 <img width=500px height=100px src="https://docs.delta.io/latest/_static/delta-lake-logo.png" alt="Project logo"></a>
</p>

<h6 align="center">Delta Lake é um projeto de código aberto que permite a construção de uma arquitetura Lakehouse. Fornecendo features como, transações ACID, manipulação de metadados escalonáveis e unificação do modo de realizar processamento de dados em lote e streaming sobre os data lakes existentes, como S3, ADLS, GCS e HDFS.</h6>

<div align="center">
</div>

<h6 align="center">
Delta Lake é uma camada de armazenamento que utiliza o formato Parquet como padrão e que fornece transações compatíveis com ACID e benefícios adicionais para Data Lakes. O Delta Lake pode ajudar a resolver diversos problemas alguns deles são:
</h6>
    
- Dificuldade em anexar dados
- Jobs extremamente custosos que falham durante a execução
- Modificações de dados armazenados são difíceis
- Operações em tempo real
- É caro manter versões históricas de dados
- Difícil de lidar com metadados grandes
- Problemas de “muitos arquivos”
- É difícil obter um ótimo desempenho
- Problemas de qualidade de dados


In [2]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable 

spark = (
        SparkSession
        .builder
        .appName("Exemplo Delta")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .enableHiveSupport()
        .getOrCreate()
)    
spark.sparkContext.setLogLevel("WARN")

In [26]:
spark

In [ ]:
spark.sparkContext.getConf().getAll()

In [ ]:
from delta.tables import DeltaTable

## Lendo dados brutos

In [4]:
path_landing = {
    "user":"./landing-zone/user"
}

In [6]:
df_user = (
    spark
    .read
    .format("json")
    .load(path_landing["user"])
    .select("user_id", "email")
)

df_user.show(n=5, truncate=False)
print(f"QTD REGISTROS: {df_user.count()}")

+-------+-----------------------+
|user_id|email                  |
+-------+-----------------------+
|1703   |daron.bailey@email.com |
|3650   |jonah.barrows@email.com|
|8809   |carla.hansen@email.com |
|4606   |tomas.ledner@email.com |
|1      |alyse.ortiz@email.com  |
+-------+-----------------------+
only showing top 5 rows

QTD REGISTROS: 800


## Escrevendo os dados em formato Delta

In [7]:
path_bronze = {
    "user":"./bronze-zone/user"
}

In [16]:
(
    df_user
    .coalesce(1)
    .write
    .format("delta")
    .mode("overwrite") # overwrite | append
    .save(path_bronze["user"])
)

## Histórico da Delta Table

In [9]:
deltaTable = DeltaTable.forPath(spark, path_bronze["user"])
deltaTable.history().show(truncate=False, vertical=True)

-RECORD 0-----------------------------------------------------------------------------
 version             | 0                                                              
 timestamp           | 2025-05-07 12:25:37.573                                        
 userId              | null                                                           
 userName            | null                                                           
 operation           | WRITE                                                          
 operationParameters | {mode -> Overwrite, partitionBy -> []}                         
 job                 | null                                                           
 notebook            | null                                                           
 clusterId           | null                                                           
 readVersion         | null                                                           
 isolationLevel      | Serializable        

## Delete

In [10]:
print("ANTES DE DELETAR")
history = deltaTable.toDF()
history.where("user_id == 1").show(truncate=False)

print("APÓS DELETAR")
deltaTable.delete(
    "user_id == 1"
)

history = deltaTable.toDF()
history.where("user_id == 1").show(truncate=False)

ANTES DE DELETAR
+-------+---------------------+
|user_id|email                |
+-------+---------------------+
|1      |alyse.ortiz@email.com|
+-------+---------------------+

APÓS DELETAR
+-------+-----+
|user_id|email|
+-------+-----+
+-------+-----+



## Update

In [17]:
print("ANTES DE ATUALIZAR")
history = deltaTable.toDF()
history.where("email == 'marcos.collier@email.com'").show(truncate=False)

print("APÓS ATUALIZAR")
deltaTable.update(
    condition = "email = 'marcos.collier@email.com'",
    set = { "email": "'unipe@gmail.com'" } 
)

history = deltaTable.toDF()
history.where("email == 'unipe@gmail.com'").show(truncate=False)

ANTES DE ATUALIZAR
+-------+------------------------+
|user_id|email                   |
+-------+------------------------+
|3395   |marcos.collier@email.com|
+-------+------------------------+

APÓS ATUALIZAR
+-------+---------------+
|user_id|email          |
+-------+---------------+
|3395   |unipe@gmail.com|
+-------+---------------+



## Gerando novos dados para serem atualizados de forma Incremental

In [18]:
items = [
        ("3395", "example@gmail.com"), 
        ("3650", "example1@gmail.com.br")
]

items

[('3395', 'example@gmail.com'), ('3650', 'example1@gmail.com.br')]

In [19]:
cols = [
        "user_id",
        "email"
]

cols

['user_id', 'email']

In [23]:
line = "____________________________________"
print(f"{line}\nNOVOS DADOS PARA ATUALIZAR")
df_new = spark.createDataFrame(items, cols)
df_new.show(truncate=False)
print(df_new.count())



print(f"{line}\nDADOS DESATUALIZADOS")
df_user.where("user_id in (3395, 3650)").orderBy("user_id").show(truncate=False)
df_user.count()

____________________________________
NOVOS DADOS PARA ATUALIZAR
+-------+---------------------+
|user_id|email                |
+-------+---------------------+
|3395   |example@gmail.com    |
|3650   |example1@gmail.com.br|
+-------+---------------------+

2
____________________________________
DADOS DESATUALIZADOS
+-------+------------------------+
|user_id|email                   |
+-------+------------------------+
|3395   |marcos.collier@email.com|
|3650   |jonah.barrows@email.com |
+-------+------------------------+



800

## Upserts

In [14]:
# !rm -rf /home/jovyan/work/datalake/s3/bronze/user/

In [24]:
(
    deltaTable.alias("desatualizados")
    .merge(
        df_new.alias("atualizados"),"desatualizados.user_id = atualizados.user_id"
    )
    .whenMatchedUpdateAll(
        condition = "desatualizados.user_id = atualizados.user_id"
    )
    .whenNotMatchedInsertAll()
    .execute()
)

## Após Upserts

In [25]:
delta_atualizado = deltaTable.toDF()

(
    delta_atualizado
    .where(
        """
        user_id 
            in (3650, 3395)
        """)
    .show(truncate=False)
)

+-------+---------------------+
|user_id|email                |
+-------+---------------------+
|3395   |example@gmail.com    |
|3650   |example1@gmail.com.br|
+-------+---------------------+



## Viagem no Tempo

In [27]:
history_delta = deltaTable.history()

In [29]:
history_delta.show(
    vertical=True,  
    truncate=False,
    n=2
)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 2                                                                                                                                                                                                                                                                                                                
 timestamp           | 2025-05-07 12:30:02.402                                                                                                                                                                                                                                                                                          
 userId      

In [32]:
#mostrando ultima escrita
(
    history_delta
    .select("version",
            "operation",
            "timestamp",
            "operationMetrics"
    )
    .where("operation == 'WRITE'")
    .show(truncate=False)
)

+-------+---------+-----------------------+--------------------------------------------------------------+
|version|operation|timestamp              |operationMetrics                                              |
+-------+---------+-----------------------+--------------------------------------------------------------+
|0      |WRITE    |2025-05-07 12:27:44.006|{numFiles -> 1, numOutputRows -> 800, numOutputBytes -> 14621}|
+-------+---------+-----------------------+--------------------------------------------------------------+



In [33]:
time_travel_version_0 = (
    spark
    .read
    .format("delta")
    .option("versionAsOf", "0")
    .load(path_bronze["user"])
)

print("VIAJANDO NO TEMPO PARA A VERSÃO 0 DA INGESTÃO")
(
    time_travel_version_0
    .where("user_id in (3650, 3395)")
    .show(truncate=False)
)

VIAJANDO NO TEMPO PARA A VERSÃO 0 DA INGESTÃO
+-------+------------------------+
|user_id|email                   |
+-------+------------------------+
|3650   |jonah.barrows@email.com |
|3395   |marcos.collier@email.com|
+-------+------------------------+



In [35]:
(
    history_delta
    .select("version",
            "operation",
            "timestamp",
            "operationMetrics"
    )
    .where("operation == 'WRITE'")
    .show(vertical=True, 
          truncate=False
    )
)

-RECORD 0--------------------------------------------------------------------------
 version          | 0                                                              
 operation        | WRITE                                                          
 timestamp        | 2025-05-07 12:27:44.006                                        
 operationMetrics | {numFiles -> 1, numOutputRows -> 800, numOutputBytes -> 14621} 



In [36]:
last_timestamp = history_delta.select("timestamp").collect()[0][0]

In [37]:
time_travel_timestamp = (
    spark
    .read
    .format("delta")
    .option("timestampAsOf", last_timestamp) # register timestamp
    .load(path_bronze["user"])
)

print("VIAJANDO NO TEMPO PARA DATA/HORA QUE HOUVE O UPSERT DOS DADOS")

time_travel_timestamp.where("user_id in (3650, 3395)").show(truncate=False)

VIAJANDO NO TEMPO PARA DATA/HORA QUE HOUVE O UPSERT DOS DADOS
+-------+---------------------+
|user_id|email                |
+-------+---------------------+
|3395   |example@gmail.com    |
|3650   |example1@gmail.com.br|
+-------+---------------------+



## Comparando modificações entre diferentes versões do Delta

In [42]:
time_travel_version_0.where("user_id in (3650, 3395)").show()

+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   3650|jonah.barrows@ema...|
|   3395|marcos.collier@em...|
+-------+--------------------+



In [38]:
time_travel_timestamp.exceptAll(time_travel_version_0).show(truncate=False)

+-------+---------------------+
|user_id|email                |
+-------+---------------------+
|3650   |example1@gmail.com.br|
|3395   |example@gmail.com    |
+-------+---------------------+



## Evolution Schema (Evolução da estrutura da Tabela)

## Concistencia do DADO (ACID) - C

In [43]:
df_user = (
    spark
    .read
    .format("json")
    .load(path_landing["user"])
    .select("user_id", "email", "gender")
)

print("*"*20)
print("NOVA COLUNA A SER INSERIDA")
df_user.printSchema()
print("*"*20)
print("ÚLTIMO ESTADO DA TABELA DELTA")
delta_atualizado.printSchema()

********************
NOVA COLUNA A SER INSERIDA
root
 |-- user_id: long (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)

********************
ÚLTIMO ESTADO DA TABELA DELTA
root
 |-- user_id: long (nullable = true)
 |-- email: string (nullable = true)



## Tentativa de Sobrescrever o dado sem evoluir o Schema (Concistência)

In [45]:
(
    df_user
    .write
    .format("delta")
    .mode("overwrite")
    .save(path_bronze["user"])
)

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 736339a6-9672-4f19-88e3-01503aec0f92).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- user_id: long (nullable = true)
-- email: string (nullable = true)


Data schema:
root
-- user_id: long (nullable = true)
-- email: string (nullable = true)
-- gender: string (nullable = true)

         
To overwrite your schema or change partitioning, please set:
'.option("overwriteSchema", "true")'.

Note that the schema can't be overwritten when using
'replaceWhere'.
         

## Realizando a evolução do Schema com a opção MergeSchema
- A opção mergeSchema vai substituir tudo que não existir no último estado com o valor padrão nulo.

In [46]:
(
    df_user
    .write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", True)
    .save(path_bronze["user"])
)

## Lendo os dados após o MergeSchema

In [47]:
df_user = (
    spark
    .read
    .format("delta")
    .load(path_bronze["user"])
    .select("user_id", "email", "gender")
)

print("*"*20)
print("ESTRUTURA DA TABELA DELTA")
df_user.printSchema()
print("*"*20)
print("VISUALIZANDO TABELA DELTA")
df_user.show()

********************
ESTRUTURA DA TABELA DELTA
root
 |-- user_id: long (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)

********************
VISUALIZANDO TABELA DELTA
+-------+--------------------+-----------+
|user_id|               email|     gender|
+-------+--------------------+-----------+
|   3395|marcos.collier@em...|   Bigender|
|   1556|elina.hills@email...| Non-binary|
|   1879|enedina.schroeder...|     Female|
|   7805|colin.ryan@email.com|       Male|
|   3982|dallas.boyle@emai...| Polygender|
|   7274|grover.towne@emai...|    Agender|
|   3184|dexter.schmitt@em...|Genderfluid|
|    550|novella.weber@ema...|Genderqueer|
|   8365|lesley.mccullough...|       Male|
|   4942|marti.marks@email...|   Bigender|
|   8327|shawnna.keebler@e...|Genderfluid|
|   9464|guillermo.beahan@...|   Bigender|
|   4123|sid.bechtelar@ema...| Non-binary|
|   2281|merrill.upton@ema...|     Female|
|   6998|felipe.ward@email...|    Agender|
|   7440|willi

## Removendo coluna e fazendo o MergeSchema novamente

In [49]:
df_user = df_user.drop("gender")
df_user.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- email: string (nullable = true)



In [50]:
(
    df_user
    .write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", True)
    .save(path_bronze["user"])
)

## Lendo os dados após o MergeSchema

In [51]:
df_user = (
    spark
    .read
    .format("delta")
    .load(path_bronze["user"])
    .select("user_id", "email", "gender")
)

print("*"*20)
print("ESTRUTURA DA TABELA DELTA")
df_user.printSchema()
print("*"*20)
print("VISUALIZANDO TABELA DELTA")
df_user.show()

********************
ESTRUTURA DA TABELA DELTA
root
 |-- user_id: long (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)

********************
VISUALIZANDO TABELA DELTA
+-------+--------------------+------+
|user_id|               email|gender|
+-------+--------------------+------+
|   5795|sudie.witting@ema...|  null|
|   3130|dalila.orn@email.com|  null|
|   8677|forest.macgyver@e...|  null|
|   4915|cyndy.hand@email.com|  null|
|   5995|darrin.kertzmann@...|  null|
|   8092|jeremy.corwin@ema...|  null|
|   4941|france.reilly@ema...|  null|
|   8658|clay.vandervort@e...|  null|
|   1907|trevor.green@emai...|  null|
|   7275|darell.prohaska@e...|  null|
|   9558| jaye.kihn@email.com|  null|
|   8276|luise.pagac@email...|  null|
|   5530|hollis.goyette@em...|  null|
|   2894|erika.turcotte@em...|  null|
|   6878|virgilio.kling@em...|  null|
|   1295|benny.bashirian@e...|  null|
|   5445|gerri.lesch@email...|  null|
|   6104|fleta.anderson@em..

## Realizando a evolução do Schema com a opção Overwrite Schema
- A opção Overwrite Schema vai gravar uma nova versão da tabela delta removendo a coluna de forma literal.

In [52]:
df_user = (
    spark
    .read
    .format("json")
    .load(path_landing["user"])
    .select("user_id","email","gender")
)

df_user = df_user.drop("gender")
print("*"*20)
print("ESTRUTURA DA TABELA DELTA")
df_user.printSchema()
print("*"*20)
print("VISUALIZANDO TABELA DELTA")
df_user.show()

********************
ESTRUTURA DA TABELA DELTA
root
 |-- user_id: long (nullable = true)
 |-- email: string (nullable = true)

********************
VISUALIZANDO TABELA DELTA
+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   1703|daron.bailey@emai...|
|   3650|jonah.barrows@ema...|
|   8809|carla.hansen@emai...|
|   4606|tomas.ledner@emai...|
|      1|alyse.ortiz@email...|
|   9245|russell.kulas@ema...|
|   3425|armida.lehner@ema...|
|   4264|tad.sanford@email...|
|   1668|rosia.jones@email...|
|    343|candy.conroy@emai...|
|   7393|dulcie.gottlieb@e...|
|   3909|rodrigo.reynolds@...|
|   9952|jenna.bode@email.com|
|   2364|dan.herman@email.com|
|   1611|stanley.witting@e...|
|   1723|clarinda.kilback@...|
|   7032|charley.carroll@e...|
|    549|cameron.harris@em...|
|   4161|reyes.stracke@ema...|
|    503|jolynn.schulist@e...|
+-------+--------------------+
only showing top 20 rows



## Sobrescrevendo a estrutura da Tabela

In [53]:
(
    df_user
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", True)
    .save(path_bronze["user"])
)

## Lendo após sobrescrita da estrutura da Tabela

In [54]:
df_user = (
    spark
    .read
    .format("delta")
    .load(path_bronze["user"])
)

print("*"*20)
print("ESTRUTURA DA TABELA DELTA")
df_user.printSchema()
print("*"*20)
print("VISUALIZANDO TABELA DELTA")
df_user.show()

********************
ESTRUTURA DA TABELA DELTA
root
 |-- user_id: long (nullable = true)
 |-- email: string (nullable = true)

********************
VISUALIZANDO TABELA DELTA
+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   5795|sudie.witting@ema...|
|   3130|dalila.orn@email.com|
|   8677|forest.macgyver@e...|
|   4915|cyndy.hand@email.com|
|   5995|darrin.kertzmann@...|
|   8092|jeremy.corwin@ema...|
|   4941|france.reilly@ema...|
|   8658|clay.vandervort@e...|
|   1907|trevor.green@emai...|
|   7275|darell.prohaska@e...|
|   9558| jaye.kihn@email.com|
|   8276|luise.pagac@email...|
|   5530|hollis.goyette@em...|
|   2894|erika.turcotte@em...|
|   6878|virgilio.kling@em...|
|   1295|benny.bashirian@e...|
|   5445|gerri.lesch@email...|
|   6104|fleta.anderson@em...|
|   6344|wilfredo.von@emai...|
|   8294|nereida.tillman@e...|
+-------+--------------------+
only showing top 20 rows



## Forma fácil de realizar uma Migração de Data Lake para Delta

In [61]:
#deltaTable = DeltaTable.convertToDelta(spark, f"parquet. `./landing-zone/legacy-parquet-user/`")
deltaTable = DeltaTable.forPath(spark, "./landing-zone/legacy-parquet-user/")

In [62]:
deltaTable.history().show(truncate=False,vertical=True)

-RECORD 0--------------------------------------------------------------------------
 version             | 0                                                           
 timestamp           | 2025-05-07 12:39:43.098                                     
 userId              | null                                                        
 userName            | null                                                        
 operation           | CONVERT                                                     
 operationParameters | {numFiles -> 4, partitionedBy -> [], collectStats -> false} 
 job                 | null                                                        
 notebook            | null                                                        
 clusterId           | null                                                        
 readVersion         | -1                                                          
 isolationLevel      | Serializable                                         

# Melhorando a performace para consulta reduzindo os small files

In [63]:
deltaTable = DeltaTable.forPath(spark, "./bronze-zone/user/")

In [64]:
deltaTable.optimize().executeCompaction()

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint>]